In [ ]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI()

In [ ]:
b = chat.predict("행성은 몇개 있습니까?")

print(b)

# Template

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate


chat = ChatOpenAI(temperature=0.1)

## Prompt template

In [ ]:
template = PromptTemplate.from_template(
    "{country_a} 과 {country_b} 거리는 어떻게 됩니까?",
)
prompt = template.format(country_a="멕시코", country_b="태국")
chat.predict(prompt)
# '멕시코와 태국 사이의 거리는 대략 17,000 킬로미터입니다.'


## Chat Prompt Template

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "안녕 만나서 반가워! 나는 {name} 이야"),
        (
            "human",
            "{country_a} 과 {country_b} 거리는 어떻게 됩니까?, 또한 당신의 이름은 무엇입니까?",
        ),
    ]
)

In [ ]:
prompt = template.format_messages(
    language="Korean",
    name="장영실",
    country_a="korea",
    country_b="japan"
)

chat.predict_messages(prompt)
# AIMessage(content='한국과 일본 사이의 거리는 대략 1,000km 정도입니다. 제 이름은 장영실입니다.')

# OutputParcer and LCEL

In [ ]:
from langchain.schema import BaseOutputParser
from langchain.prompts import ChatPromptTemplate


class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

In [ ]:
p = CommaOutputParser()
p.parse("Hello, How, are, you?")

In [ ]:
template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Your are a list generating machine. Everything you are asked will be answered with a comma sepreated list of max {max_items}. Do not reply with anything else.",
        ),
        ("human", "{question}"),
    ]
)

In [ ]:
prompt = template.format_messages(
    max_items = 10,
    question="What are the planets?"
)

result = chat.predict_messages(prompt)

In [ ]:
p = CommaOutputParser()
p.parse(result.content)

## Chain

In [ ]:
chain = template | chat | CommaOutputParser()

In [ ]:
chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?"
})

# Chaining Chains

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [ ]:
chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "Your are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

In [ ]:
chef_chain = chef_prompt | chat

In [ ]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")                                              
])

In [ ]:
veg_chain = veg_chef_prompt | chat

In [ ]:
final_chain = {"recipe": chef_chain} | veg_chain

In [ ]:
final_chain.invoke({
    "cuisine": "indian"
})